In [1]:
import os, sys
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns 

from os.path import join 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from IPython.display import display_html

import warnings
warnings.filterwarnings("ignore")

In [3]:
DataPath = r'D:\GitWork\titanic\data'
OutPath  = r'D:\GitWork\titanic\output'

f_abspath = join(DataPath, 'train.csv')
df_train = pd.read_csv(f_abspath)
display(df_train.head())
display(df_train.describe())

f_abspath = join(DataPath, 'test.csv')
df_test = pd.read_csv(f_abspath)
display(df_test.head())
display(df_test.describe())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [39]:
# Preprocess Pclass
features = ['PassengerId', 'Survived', 'Sex', 'Pclass', 'Fare']
df_data = df_train[features]

# For Sex mapping
le = LabelEncoder()
df_data.insert(3, 'Sex_Code', le.fit_transform(df_data.Sex))
display(df_data.head())

# For fare mapping
df_data['Bin4'] = pd.qcut(df_data.Fare, 4)
df_data['Bin4_Code'] = le.fit_transform(df_data.Bin4)

print('\nBin4 classes dict:')

def displayLeDict(le):
    dic = {
        le.transform([x])[0]: x for x in le.classes_
    }
    display(dic)
    return

displayLeDict(le)

indexes = [df_data.Pclass, df_data.Bin4_Code]
columns = [df_data.Survived]
kwargs = { 'margins': True, 'margins_name': 'Total' }
ct = pd.crosstab(indexes, columns, values=df_data.Fare, aggfunc='mean').round(4)
display(ct)

,PassengerId,Survived,Sex,Sex_Code,Pclass,Fare
0,1,0,male,1,3,7.2500
1,2,1,female,0,1,71.2833
2,3,1,female,0,3,7.9250
3,4,1,female,0,1,53.1000
4,5,0,male,1,3,8.0500



Bin4 classes dict:


{0: Interval(-0.001, 7.91, closed='right'),
 1: Interval(7.91, 14.454, closed='right'),
 2: Interval(14.454, 31.0, closed='right'),
 3: Interval(31.0, 512.329, closed='right')}

Survived                0         1
Pclass Bin4_Code                   
1      0           0.8333       NaN
       2          27.9075   28.0449
       3          89.9988  112.3438
2      0           0.0000       NaN
       1          12.1892   12.3292
       2          24.3621   24.0374
       3          55.4833   41.8677
3      0           7.3967    7.4629
       1           9.0700    9.6964
       2          20.4110   18.2909
       3          45.6721   47.0802

In [32]:
columns = [df_data.Pclass, df_data.Bin4_Code, df_data.Survived]
sector = df_data.Fare.groupby(columns).mean()
display(sector)

# df_data['PclassBin_Code'] = le.fit_transform(df_data.Bin4)
# display(df_data.head(10))

Pclass  Bin4_Code  Survived
1       0          0             0.833333
        2          0            27.907458
                   1            28.044907
        3          0            89.998832
                   1           112.343848
2       0          0             0.000000
        1          0            12.189151
                   1            12.329167
        2          0            24.362050
                   1            24.037400
        3          0            55.483340
                   1            41.867717
3       0          0             7.396727
                   1             7.462884
        1          0             9.070025
                   1             9.696431
        2          0            20.410991
                   1            18.290878
        3          0            45.672094
                   1            47.080188
Name: Fare, dtype: float64

In [ ]:
# Cross table
ct = pd.crosstab(df_tmp.Pclass, df_tmp.Survived, margins=True, margins_name='Total')
ct['S1_Rate'] = (ct[1] / ct.Total).round(3)
display(ct)

df_plot = pd.DataFrame({
    'dead' : df_tmp.Pclass[df_tmp.Survived==0].value_counts(),
    'alive': df_tmp.Pclass[df_tmp.Survived==1].value_counts()
})

fig, ax = plt.subplots(figsize=(8,5))
plotBarUnstack(df_plot, ax, 'Pclass & Survived', 'Pclass', 'Count')
plt.show()